## 安裝套件

In [ ]:
!pip install finlab > log.txt
!pip install ta-lib-bin > log.txt

## 營建產業限定策略

運用營建股"合約負債"的特性進行限定產業選股。

[詳細文章內容](https://www.finlab.tw/building-contingent-liability-strategy/)

In [ ]:
from finlab import data
from finlab.backtest import sim
import pandas as pd

# 限定回測產業範圍
with data.universe(category='建材營造'):

    # 資料引入
    close = data.get('price:收盤價')
    contract_debt = data.get('financial_statement:合約負債_流動')
    contract_debt_gr = (contract_debt/(contract_debt.shift())-1)
    equity = data.get('financial_statement:股本')

    # 計算合約負債佔股本的比率，比率越高代表未來潛在貢獻每股營收越高
    ce_ratio = (contract_debt / equity)

    # 選股條件
    cond1 = ce_ratio > 0.5
    cond2 = (contract_debt_gr > 0.05) & (contract_debt_gr<0.5)
    cond_all = cond1 & cond2

    result = ce_ratio * cond_all
    position = result[result > 0].is_largest(5)
    
    # position_limit 限定單檔標的最多持有30%部位，避免標的數過少時產生重壓的非系統性風險
    report = sim(position,resample='M',position_limit=0.3,stop_loss=0.1,fee_ratio=1.425/1000*0.3,name="營建股合約負債策略",upload=False)

report.display()

stock_id
2540    0.2
2548    0.2
3056    0.2
5213    0.2
6186    0.2
Name: 2022-05-31 00:00:00, dtype: float64

Timestamp('2022-05-31 00:00:00')